In [30]:
!pip install -q pyomo

In [31]:
from pyomo.environ import *

In [32]:
import numpy as np

In [33]:
model = ConcreteModel()

In [34]:
M=4
N=10

In [35]:
obj_coef_c = np.array([-9.6, -2.85, 41.5, 33.05, -0.75, 20.5, -0.35, -31.02, 50.65, 6.125])

In [36]:
constr_coef_A= np.array(([0,1,-2,0,1,-2,0,1,1,-1],[-1,1,-3,2.5,0, 1.4, 0, -2, 1, -1.5],[-0.45, 4.78, -2, -3.22, -1.3, 1, -1, 0, 0, 0.25],[1 ,0, 0, 0 ,1, 1 ,0, 2 ,3, 0]))

In [37]:
constr_rhs_b = np.array([35.4, 47.9, 41.43, 22.7])

In [38]:
lower_bound = np.array([-np.inf, -np.inf, 0, 0, -67, 0, -23, -5, 6, 1])
upper_bound = np.array([4, np.inf, np.inf, 2, 20, 55, 23, 15, np.inf, np.inf])

In [39]:
col_indices = np.arange(N)
print(col_indices)
print(col_indices[3])
row_indices=np.arange(M)
print(row_indices)

[0 1 2 3 4 5 6 7 8 9]
3
[0 1 2 3]


In [40]:
model.x = Var(col_indices)

In [41]:
model.constraints = ConstraintList()

In [42]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    0 :      {}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

1 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  None :  None :  None : False :  True :  Reals
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals
          3 :  None :  None :  None : False :  True :  Reals
          4 :  None :  None :  None : False :  True :  Reals
          5 :  None :  None :  None : False :  True :  Reals
          6 :  None :  None :  None : False :  True :  Reals
          7 :  None :  None :  None : False :  True :  Reals
          8 :  None :  None :  None : False :  True :  Reals

In [43]:
for i in row_indices :
  model.constraints.add(sum(constr_coef_A[i,j]*model.x[j] for j in col_indices) <= constr_rhs_b[i])


In [44]:
for j in col_indices:
  model.x[j].setlb(lower_bound[j])
  model.x[j].setub(upper_bound[j])

In [45]:
model.objective = Objective(expr = sum(obj_coef_c[j]*model.x[j] for j in col_indices), sense=minimize)

In [46]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

1 Var Declarations
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  -inf :  None :   4.0 : False :  True :  Reals
          1 :  -inf :  None :   inf : False :  True :  Reals
          2 :   0.0 :  None :   inf : False :  True :  Reals
          3 :   0.0 :  None :   2.0 : False :  True :  Reals
          4 : -67.0 :  None :  20.0 : False :  True :  Reals
          5 :   0.0 :  None :  55.0 : False :  True :  Reals
          6 : -23.0 :  None :  23.0 : False :  True :  Reals
          7 :  -5.0 :  None :  15.0 : False :  True :  Reals
          8 :   6.0 :  None :   inf : False :  True :  

In [47]:
!apt-get install -y -qq glpk-utils

In [48]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -196.378974895398
  Upper bound: -196.378974895398
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 11
  Number of nonzeros: 29
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.013497352600097656
# ----------------------------------------------------------
#   Solution Information
# ----------

In [49]:
# display solution
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)

print('\nConstraints')
model.constraints.display()


Objective =  -196.3789748953975

Decision Variables
x[ 0 ] =  4.0
x[ 1 ] =  5.8347280334728
x[ 2 ] =  0.0
x[ 3 ] =  0.0
x[ 4 ] =  -29.3
x[ 5 ] =  0.0
x[ 6 ] =  23.0
x[ 7 ] =  15.0
x[ 8 ] =  6.0
x[ 9 ] =  1.0

Constraints
constraints : Size=4
    Key : Lower : Body               : Upper
      1 :  None : -3.465271966527201 :  35.4
      2 :  None :  -23.6652719665272 :  47.9
      3 :  None :  41.42999999999998 : 41.43
      4 :  None :               22.7 :  22.7
